# New York CitiBike Data Science Challenge

Der Fahrradverleih CitiBike vermietet in New York über 12.000 Fahrräder an 750 
Verleihstationen. Somit ist CitiBike eine echte Alternative zu den herkömmlichen 
Transportmitteln, wie z.B. U-bahn oder Taxi. 

Nehme an, es wäre das Jahr 2018. Mit einem gültigen 24 Stunden (3 Tage) Pass bzw. einer 
jährlichen Mitgliedschaft können Kunden ein Fahrrad an einer Verleihstation abholen und an 
einer beliebigen Station wieder abgeben. CitiBike stellt die durch den Verleih gesammelten 
Daten der Öffentlichkeit zur Verfügung. Deine Aufgabe als Data Scientist ist es, CitiBike dabei zu
helfen diese Daten wertstiftend zu nutzen. 

In einem ersten Pilotprojekt sollst du hierfür die Daten analysieren und ein Modell bauen, mit 
welchem zwei Klassen von Nutzern identifiziert werden können; dies sind (i) customers (24 
Stunden/ 3 Tage Pass) und (ii) subscribers (jährliche Mitgliedschaft).

Eine Orientierungshilfe bei 
dieser Aufgabenstellung bieten dir die folgenen Arbeitsschritte:
 
1. Lade diese Daten von Citibike für das Jahr 2018 herunter. Mach dich mit dem Inhalt des 
Datensatzes vertraut und bereite ihn für weitere Analysen auf. 
2. Visualisiere die Daten; sei kreativ und überlege dir geeignete Darstellungsformen für 
deine Entdeckungen. Nutze die Visualisierungen auch in deiner Ergebnispräsentation um 
deine Argumente zu unterstützen.
3. Überlege dir geeignete Features (Merkmale) als Input für dein customer-subscriber-
Modell. Konstruiere gegebenenfalls neue Features um dein Modell zu verbessern. 
4. Verteste verschiedene Modelle bzw. Methoden zur Klassifikation der Kundentypen 
subscribers und customers. Evaluiere die Performance der unterschiedlichen Modelle 
und begründe eine Modellauswahl.
5. Skizziere mögliche Einsatzgebiete oder UseCases deiner Modelle
6. Skizziere für CitiBike Kooperationsmöglichkeiten mit einer Versicherung (und/oder 
umgekehrt). 
 


## Imports und Co

In [1]:
# Generelles
import os

# Data science
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# Plotting
import matplotlib.pyplot as plt

# Hilfen
from utils import crow_distance

# Display
pd.set_option('display.max_columns', 500)
%matplotlib inline 

# Random seed
np.random.seed(1)

## Datenvorbereitung

In [2]:
# Rohdaten einlesen, wir beschränken uns auf die Sommermonate um meine virtuelle Maschine zu schonen
data_directory = 'data_NY/'
files = sorted(os.listdir(data_directory))
for index, file in enumerate(files):
    if index == 0:
        data = pd.read_csv(data_directory+file)
    else:
        data_tmp = pd.read_csv(data_directory+file)
        data = pd.concat([data, data_tmp], ignore_index=True)

# Datentyp verbessern
for column in data.columns:
    data.rename(columns={column: column.replace(' ', '_')}, inplace=True)
data.starttime = pd.to_datetime(data.starttime)
data.stoptime = pd.to_datetime(data.stoptime)

# Rohdaten sichten
data

,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,569,2018-06-01 01:57:20.514,2018-06-01 02:06:50.088,72.0,W 52 St & 11 Ave,40.767272,-73.993929,173.0,Broadway & W 49 St,40.760683,-73.984527,21481,Subscriber,1999,1
1,480,2018-06-01 02:02:42.398,2018-06-01 02:10:43.354,72.0,W 52 St & 11 Ave,40.767272,-73.993929,477.0,W 41 St & 8 Ave,40.756405,-73.990026,19123,Subscriber,1988,1
2,692,2018-06-01 02:04:23.624,2018-06-01 02:15:55.747,72.0,W 52 St & 11 Ave,40.767272,-73.993929,457.0,Broadway & W 58 St,40.766953,-73.981693,26983,Subscriber,1986,1
3,664,2018-06-01 03:00:55.461,2018-06-01 03:11:59.906,72.0,W 52 St & 11 Ave,40.767272,-73.993929,379.0,W 31 St & 7 Ave,40.749156,-73.991600,26742,Subscriber,1973,1
4,818,2018-06-01 06:04:54.427,2018-06-01 06:18:32.617,72.0,W 52 St & 11 Ave,40.767272,-73.993929,459.0,W 20 St & 11 Ave,40.746745,-74.007756,26386,Subscriber,1984,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5843900,592,2018-08-31 23:59:27.653,2018-09-01 00:09:20.580,2003.0,1 Ave & E 18 St,40.733812,-73.980544,518.0,E 39 St & 2 Ave,40.747804,-73.973442,33086,Subscriber,1996,1
5843901,451,2018-08-31 23:59:42.840,2018-09-01 00:07:14.533,249.0,Harrison St & Hudson St,40.718710,-74.009001,2008.0,Little West St & 1 Pl,40.705693,-74.016777,33066,Subscriber,1960,1
5843902,472,2018-08-31 23:59:48.531,2018-09-01 00:07:41.041,450.0,W 49 St & 8 Ave,40.762272,-73.987882,281.0,Grand Army Plaza & Central Park S,40.764397,-73.973715,28711,Subscriber,1966,1
5843903,1239,2018-08-31 23:59:50.762,2018-09-01 00:20:30.457,3107.0,Bedford Ave & Nassau Ave,40.723117,-73.952123,372.0,Franklin Ave & Myrtle Ave,40.694546,-73.958014,33935,Subscriber,1984,1


In [3]:
# Sanity checks der Rohdaten

print('Tripduration')
print(f'Min: {np.min(data.tripduration)} s, Max: {np.max(data.tripduration)/3600:.0f} h\n')

print('Starttime')
print(f'Min: {np.min(data.starttime)}, Max: {np.max(data.starttime)}\n')

print('Stoptime')
print(f'Min: {np.min(data.stoptime)}, Max: {np.max(data.stoptime)}\n')

print('Start station id')
print(f'Min: {np.min(data.start_station_id)}, Max: {np.max(data.start_station_id)}, #: {len(np.unique(data.start_station_id))}\n')

# print('Start station name')
# print(f'#: {len(np.unique(data.start_station_name))}\n')
      
print('Start station latitude')
print(f'Min: {np.min(data.start_station_latitude)}, Max: {np.max(data.start_station_latitude)}, #: {len(np.unique(data.start_station_latitude))}\n')

print('Start station longtude')
print(f'Min: {np.min(data.start_station_longitude)}, Max: {np.max(data.start_station_longitude)}, #: {len(np.unique(data.start_station_longitude))}\n')

print('End station id')
print(f'Min: {np.min(data.end_station_id)}, Max: {np.max(data.end_station_id)}, #: {len(np.unique(data.end_station_id))}\n')

# print('End station name')
# print(f'#: {len(np.unique(data.end_station_name))}\n')

print('End station latitude')
print(f'Min: {np.min(data.end_station_latitude)}, Max: {np.max(data.end_station_latitude)}, #: {len(np.unique(data.end_station_latitude))}\n')

print('End station longitude')
print(f'Min: {np.min(data.end_station_longitude)}, Max: {np.max(data.end_station_longitude)}, #: {len(np.unique(data.end_station_longitude))}\n')

print('Bikeid')
print(f'Min: {np.min(data.bikeid)}, Max: {np.max(data.bikeid)}, #: {len(np.unique(data.bikeid))}\n')

print('Usertype')
print(f'Types: {np.unique(data.usertype)}, #: {np.unique(data.usertype, return_counts=True)[1]}\n')

print('Birth year')
print(f'Min: {np.min(data.birth_year)}, Max: {np.max(data.birth_year)}\n')

print('Gender')
print(f'Types: {np.unique(data.gender)}, #: {np.unique(data.gender, return_counts=True)[1]}\n')

Tripduration
Min: 61 s, Max: 2103 h

Starttime
Min: 2018-06-01 00:00:02.458000, Max: 2018-08-31 23:59:55.733000

Stoptime
Min: 2018-06-01 00:03:32.708000, Max: 2018-09-12 13:53:04.297000

Start station id
Min: 72.0, Max: 3705.0, #: 787

Start station latitude
Min: 40.64653836709648, Max: 45.506364054011385, #: 797

Start station longtude
Min: -74.02535319328308, Max: -73.56890559196472, #: 801

End station id
Min: 72.0, Max: 3705.0, #: 811

End station latitude
Min: 40.64653836709648, Max: 45.506364054011385, #: 821

End station longitude
Min: -74.0836394, Max: -73.56890559196472, #: 825

Bikeid
Min: 14529, Max: 34839, #: 13313

Usertype
Types: ['Customer' 'Subscriber'], #: [ 816915 5026990]

Birth year
Min: 1885, Max: 2002

Gender
Types: [0 1 2], #: [ 617831 3820766 1405308]



In [4]:
# Daten säubern

# Sortiere Fahrten aus, die länger als 2 Stunden dauerten
len_before = len(data)
data.drop(data[data.tripduration >= 2*3600].index, inplace=True)
len_after = len(data)
print(f'Dropped {len_before-len_after} lines due to excessive tripduration')

# Macht es Sinn, Geburtsjahre vor 1918 auszusortieren?

Dropped 21563 lines due to excessive tripduration


### Erstes Fazit

- Datensatz sehr groß, wir beschränken uns deswegen auf die Sommermonate
- Datensatz ist 'imbalanced', ~5 Millionen Subscriber-Fahrten gegen ~800 000 Customer-Fahrten
- Datenqualität ist okay, aber ein paar sehr alte Kunden, lange Fahrten und unhomogene Koordinatenangaben
- Wir sollten schon vor der weiteren Analyse einen Teil der Daten beiseitelegen, um eine wirklich unabhängige Evaluation unseres Modells zu erreichen
- Wir sollten schon bei der visuellen Analyse des Datensatzes die Aufgabenstellung im Auge behalten

## Features für Customer-Subscriber-Modell

In [ ]:
data['start_hour_of_day']= [t.hour for t in data.starttime]

data['start_day_of_week'] = [t.dayofweek for t in data.starttime]

data['start_day_of_year'] = [t.dayofyear for t in data.starttime]

data['duration_seconds'] = data.tripduration

data['crow_distance_meters'] = crow_distance(data.start_station_latitude, data.start_station_longitude, data.end_station_latitude, data.end_station_longitude)

data['crow_velocity_meters_per_second'] = data.crow_distance_meters / data.duration_seconds

data['age'] = 2018 - data.birth_year

data['gender_male'] = [1 if g==1 else 0 for g in data.gender]

data['gender_female'] = [1 if g==2 else 0 for g in data.gender]

data['gender_unknown'] = [1 if g==0 else 0 for g in data.gender]

data['customer'] = [1 if u=='Customer' else 0 for u in data.usertype]

data['subscriber'] = [1 if u=='Subscriber' else 0 for u in data.usertype]

data['set'] = [np.random.choice(['train', 'dev', 'test'], p=[0.8, 0.1, 0.1]) for r in range(len(data))]

data['train_set'] = [True if s=='train' else False for s in data.set]

data['dev_set'] = [True if s=='dev' else False for s in data.set]

data['test_set'] = [True if s=='test' else False for s in data.set]

data

## Visualisierungen

In [ ]:
# histogramme, sortiert nach customer und Subscriber
plot_specs = [('start_hour_of_day', range(25)),
              ('start_day_of_week', range(7)),
              ('start_day_of_year', np.linspace(140, 250, num=20)),
              ('duration_seconds', np.linspace(0, 3600, num=20)),
              ('crow_distance_meters', np.linspace(0, 6000, num=20)),
              ('crow_velocity_meters_per_second', np.linspace(0, 6, num=20)),
              ('age', np.linspace(0, 80, num=20)),
              ('gender_male', range(3)),
              ('gender_female', range(3)),
              ('gender_unknown', range(3))]

data_customer = data[~data.test_set & data.customer==1]
data_subscriber = data[~data.test_set & data.subscriber==1]

for column, bins in plot_specs:
    plt.hist([data_customer[column], data_subscriber[column]], bins, stacked=True, rwidth=.95, color=['tomato', 'silver'])
    plt.title(f'{column} histogram')
    plt.show()
    plt.close()

In [ ]:
# Wegstrecken, ebenfalls nach customer und subscriber 

for idx, row in data_customer.iterrows():
    plt.plot([row.start_station_longitude, row.end_station_longitude], [row.start_station_latitude, row.end_station_latitude], alpha=0.01, color='tomato')
plt.xlim(-74.1, -73.96)
plt.ylim(40.705, 40.825)
plt.title('customer frequent routes')
plt.show()
plt.close()

for idx, row in data_subscriber.iterrows():
    plt.plot([row.start_station_longitude, row.end_station_longitude], [row.start_station_latitude, row.end_station_latitude], alpha=0.001, color='silver')
plt.xlim(-74.1, -73.96)
plt.ylim(40.705, 40.825)
plt.title('subscriber frequent routes')
plt.show()
plt.close()

## Modelle

In [ ]:
# Erstes simples Modell: klassifiziere immer als Subscriber

def 

## Use cases für Modelle

- ...

##  Kooperation mit Versicherung

- ...

## Was ich noch gemacht hätte, wenn ich Zeit gehabt hätte

- Datensatz als Graph betrachten, zwischen welchen Stationen fahren Customer und Subscriber gerne?
- ...